# 02 Combine the Disney Films List with IMDB Data

## 02.01 Python Imports

In [1]:
import gzip
import pandas as pd

## 02.02 IMDB Title Basic Info

### 02.02.01 Import IMDB Title data

In [9]:
bs=gzip.open('../Other Source Data/IMDB/title.basics.tsv.gz','rb')
df_basics = pd.read_csv(bs,sep='\t', low_memory=False)

In [10]:
df_basics.shape

(8784772, 9)

In [11]:
df_basics.head(10)

tconst titleType                                 primaryTitle  \
0  tt0000001     short                                   Carmencita   
1  tt0000002     short                       Le clown et ses chiens   
2  tt0000003     short                               Pauvre Pierrot   
3  tt0000004     short                                  Un bon bock   
4  tt0000005     short                             Blacksmith Scene   
5  tt0000006     short                            Chinese Opium Den   
6  tt0000007     short  Corbett and Courtney Before the Kinetograph   
7  tt0000008     short       Edison Kinetoscopic Record of a Sneeze   
8  tt0000009     short                                   Miss Jerry   
9  tt0000010     short                          Leaving the Factory   

                                 originalTitle isAdult startYear endYear  \
0                                   Carmencita       0      1894      \N   
1                       Le clown et ses chiens       0      1892      \N   
2                               Pauvre Pierrot       0      1892      \N   
3                                  Un bon bock       0      1892      \N   
4                             Blacksmith Scene       0      1893      \N   
5                            Chinese Opium Den       0      1894      \N   
6  Corbett and Courtney Before the Kinetograph       0      1894      \N   
7       Edison Kinetoscopic Record of a Sneeze       0      1894      \N   
8                                   Miss Jerry       0      1894      \N   
9          La sortie de l'usine Lumière à Lyon       0      1895      \N   

  runtimeMinutes                    genres  
0              1         Documentary,Short  
1              5           Animation,Short  
2              4  Animation,Comedy,Romance  
3             12           Animation,Short  
4              1              Comedy,Short  
5              1                     Short  
6              1               Short,Sport  
7              1         Documentary,Short  
8             40             Romance,Short  
9              1         Documentary,Short

We dont want to include any films "startYear" prior to 1936 when Snow White and The Seven Dwarfs was released.  We also don't need to include any films "titleType" other than "movie" as we are only worried about theatrical releases.  However by excluding these things now, we may end up having look through more data later, but we will also avoid any false positives and some duplicates when we do the matching later. Since startYear is an object type, and there are too many values to see them all, we'll just leave it be for now.

## 02.03 Disney Film List

### 02.03.01 Import Disney Film List

In [12]:
FL_Disney = pd.read_csv('../Other Source Data/DD_Films_List_Disney_Com/Film_list_Disney_com.csv')
FL_Disney.rename(     columns=({ 'Movie Title': 'title'}),     inplace=True )

print(FL_Disney.shape)
FL_Disney.head()

(657, 1)


title
0                               101 Dalmatians
1                        101 Dalmatians (1996)
2  101 Dalmatians II: Patch's London Adventure
3                               102 Dalmatians
4                 20,000 Leagues Under the Sea

We ran in to a lot of title issues originally.  It will be faster to create lowercase, stripped down title columns, and join on those.

In [13]:
df_basics['lower_title'] = df_basics['primaryTitle'].str.lower().replace(r'[^A-Za-z0-9 ]+', '', regex=True)

In [14]:
FL_Disney['lower_title'] = FL_Disney['title'].str.lower().replace(r'[^A-Za-z0-9 ]+', '', regex=True)

In [2]:
#Double checking an issue we ran in to later down the line

In [3]:
df_basics[df_basics['tconst']=='tt0074968']

NameError: name 'df_basics' is not defined

In [4]:
FL_Disney[FL_Disney['lower_title'].str.contains("deposit")]

NameError: name 'FL_Disney' is not defined

## 02.04 Merge The Disney and IMDB Data

This is just a first pass to make sure the merge is going to work as we expect, find all of the correct titles, explore any duplicates or missing values, etc.

In [18]:
FL_Disney_IMDB = pd.merge(FL_Disney ,                 # left df
                          df_basics[['tconst','lower_title','titleType','startYear']],                  # right df
                          how="left",                 # left join
                          left_on='lower_title',            # left column
                          right_on='lower_title',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

## 02.05 Explore Combined Disney and IMDB Data

In [5]:
FL_Disney_IMDB.head()

NameError: name 'FL_Disney_IMDB' is not defined

In [6]:
print(FL_Disney_IMDB.shape)
FL_Disney_IMDB['_merge'].value_counts()

# Original results for this cell
# (6143, 11)
# both          6018
# left_only      125
# right_only       0

# 1st update  results for this cell
# (6293, 12)
# both          6209
# left_only       84
# right_only       0
# Name: _merge, dtype: int64

# 2nd update  results for this cell
# (6584, 6)
# both          6502
# left_only       82
# right_only       0
# Name: _merge, dtype: int64

# So there is more duplication, but fewer 'left only' cells, meaning it foudn more matches

NameError: name 'FL_Disney_IMDB' is not defined

In [21]:
FL_Disney_IMDB['titleType'].value_counts()


#  Original results for this cell
# tvEpisode       3675
# movie            904
# short            668
# tvMovie          244
# video            218
# tvSeries         160
# videoGame         83
# tvMiniSeries      56
# tvShort            6
# tvSpecial          4


tvEpisode       3973
movie            974
short            736
tvMovie          262
video            232
tvSeries         170
videoGame         86
tvMiniSeries      59
tvShort            6
tvSpecial          4
Name: titleType, dtype: int64

We started with 657 rows in the Disney Film list, and ended with 6502 after the joined the tables, so there are a lot of duplicates that we'll need to clean up.  82 title from the original Disney list were not found in the IMDM list. To clean this up, we'll look at the 82 from the Disney list, and explore those seperately.  

We'll pull out any single matches from the list, and explore those under the hypothesis that they are correct but will look to disprove that hypothesis.  

Finally, we'll look at the duplicated values from the list and attempt to ID which titles we really need to keep.  

The goal is to get back down to the original 657 row with complete IMDB data.  Since we only want to consider feature releases, not TV or Video, then we will likely end up with fewer than the original 657 AFTER we've done some filtering.

## 02.06 Films missing from IMDB

### 02.06.01 Identify and explore titles not found on IMDB

First, lets look at the 82 that were not found in IMDB at all. We're going to create a new column for 'updated_title' and correct that as we go.  We are operating under the assumption that as we pull in future datasets, we will run in to more name issues and will not always have the 'tconst' in each data set to rely on.  This will allow us to have multiple name options to join on later if we choose, and allow us to not alter the titles as we originally pulled them.

In [22]:
FL_Disney_IMDB[FL_Disney_IMDB['tconst'].isnull()].head(35)

title  \
144                     A Very Playhouse Disney Holiday   
318                        Aladdin: The Return of Jafar   
498                            America's Heart and Soul   
570                                        Annie (1999)   
642                                   Atta Girl, Kelly!   
1587  Chronicles of Narnia: The Lion, the Witch and ...   
1869  Disney Animation Collection Volume 1: Mickey A...   
1870  Disney Animation Collection Volume 2: Three Li...   
1871  Disney Animation Collection Volume 3: The Prin...   
1872  Disney Animation Collection Volume 4: The Tort...   
1873  Disney Animation Collection Volume 5: Wind In ...   
1874  Disney Animation Collection Volume 6: The Relu...   
1875  Disney Animation Collection Volume 7: Mickey's...   
1877                              Disney Junior Holiday   
1878  Disney Learning Adventures: Mickey And The Bea...   
1879  Disney Learning Adventures: Mickey's Around Th...   
1881                  Disney Princess Party: Volume One   
1883  Disney Princess Sing Along Songs Volume One: O...   
1884  Disney Princess Sing Along Songs Volume Three:...   
1885  Disney Princess Sing Along Songs Volume Two: E...   
1889            Disney Sing Along Songs: Happy Haunting   
1891  Disney Sing-Along-Songs: Beach Party at Walt D...   
1896                          Disney's American Legends   
1898                            Doc McStuffins: Pet Vet   
1902                      Dr. Seuss' Horton Hears a Who   
1903   DuckTales: The Movie - Treasure of the Lost Lamp   
1986    Elfego Baca And The Swamp Fox, Legendary Heroes   
2256                               Freaky Friday (1976)   
2326                        Frozen (Sing-Along Edition)   
2424  Hannah Montana & Miley Cyrus: Best of Both Wor...   
2425                   Hannah Montana: Pop Star Profile   
2434                 Hans Brinker, Or The Silver Skates   
2436             Have Yourself A Goofy Little Christmas   
2551                         High School Musical: China   
2552                   High School Musical: The Concert   

                                            lower_title tconst titleType  \
144                     a very playhouse disney holiday    NaN       NaN   
318                         aladdin the return of jafar    NaN       NaN   
498                             americas heart and soul    NaN       NaN   
570                                          annie 1999    NaN       NaN   
642                                     atta girl kelly    NaN       NaN   
1587  chronicles of narnia the lion the witch and th...    NaN       NaN   
1869  disney animation collection volume 1 mickey an...    NaN       NaN   
1870  disney animation collection volume 2 three lit...    NaN       NaN   
1871  disney animation collection volume 3 the princ...    NaN       NaN   
1872  disney animation collection volume 4 the torto...    NaN       NaN   
1873  disney animation collection volume 5 wind in t...    NaN       NaN   
1874  disney animation collection volume 6 the reluc...    NaN       NaN   
1875  disney animation collection volume 7 mickeys c...    NaN       NaN   
1877                              disney junior holiday    NaN       NaN   
1878  disney learning adventures mickey and the bean...    NaN       NaN   
1879  disney learning adventures mickeys around the ...    NaN       NaN   
1881                   disney princess party volume one    NaN       NaN   
1883  disney princess sing along songs volume one on...    NaN       NaN   
1884  disney princess sing along songs volume three ...    NaN       NaN   
1885  disney princess sing along songs volume two en...    NaN       NaN   
1889             disney sing along songs happy haunting    NaN       NaN   
1891  disney singalongsongs beach party at walt disn...    NaN       NaN   
1896                           disneys american legends    NaN       NaN   
1898                             doc mcstuffins pet vet    NaN       NaN   
1902                     

Spot checking the first 5:
 - A Very Playhouse Disney Holiday, NOT on IMDB.com, looks like a DVD release, 2005, NR, 56m <mark>We want to consider feature films. Not video releases.</mark>
 - Aladdin: The Return of Jafar: Appears on IMDB.com as "Aladdin 2: The Return of Jafar" but video, not a feature film, 1994, G, 1h 9m #tt0107952 <mark>We want to consider feature films. Not video releases.</mark>
 - America's Heart and Soul is in IMDB but as America's Heart & Soul, with an ampersand. <mark>This will be changed in the new updated_title.</mark>
 - Annie (1999) is also in IMDB as Annie, however, it is an episode of The Wonderful World of Disney and not considered a feature release. <mark>We want to consider feature films. Not TV movies. </mark>
 - Atta Girl, Kelly! is also in IMDB.cim as Atta Girl, Kelly!: Love Is Blind, however, it is an episode of The Wonderful World of Disney and not considered a feature release. <mark>We want to consider feature films. Not TV movies. </mark>
 
Disney Animated Collection:
 - These all appear to be made for TV <mark>The titleType will all need to be  updated. </mark>
 
Disney Sing Along:
 - These all appear to be made for video <mark>The titleType will all need to be  updated. </mark> 

For the sake of time, we will not be updating any rows that we intend to use in our analysis.  That means we will not be updating any of the following:
tvEpisodes, 
shorts, 
tvMovies,
videos ,
tvSeries,
videoGames ,
tvMiniSeries,
tvShorts,
tvSpecials

### 02.06.02 Correct titles not found on IMDB

In [23]:
FL_Disney_IMDB.at[498,'updated_title'] = "America's Heart And Soul"    # America'S Heart And Soul tt0381006
FL_Disney_IMDB.at[498,'tconst'] = "tt0381006"    # America'S Heart And Soul tt0381006

Since we kept the index values from the Disney list, we could use the index to update all of the "titleType"s for the TV movies or video movies <br>
FL_Disney_IMDB.at[141,'titleType'] = 'tvMovie' <BR>
However, for the sake of take, we won't bother updating any titles that we don't intend to use in our analysis.  We will scan the new list of 84 titles for feature releases and update them in the temp df. 

In [24]:
FL_Disney_IMDB.at[570,'updated_title'] = "Annie"               # Annie (1999) tt0207972
FL_Disney_IMDB.at[570,'tconst'] = "tt0207972"               # Annie (1999) tt0207972

FL_Disney_IMDB.at[1587,'updated_title'] = "The Chronicles of Narnia: The Lion, the Witch and the Wardrobe" # Chronicles Of Narnia: The Lion, The Witch And  tt0363771
FL_Disney_IMDB.at[1587,'tconst'] = "tt0363771" # Chronicles Of Narnia: The Lion, The Witch And  tt0363771

In [25]:
FL_Disney_IMDB.at[1896,'updated_title'] = "American Legends"   # Disney'S American Legends tt0372866
FL_Disney_IMDB.at[1896,'tconst'] = "tt0372866"   # Disney'S American Legends tt0372866

In [26]:
FL_Disney_IMDB.at[1903,'updated_title'] = "DuckTales the Movie: Treasure of the Lost Lamp" # 1791	Ducktales: The Movie - Treasure Of The Lost Lamp	tt0099472
FL_Disney_IMDB.at[1903,'tconst'] = "tt0099472" # 1791	Ducktales: The Movie - Treasure Of The Lost Lamp	tt0099472

In [27]:
FL_Disney_IMDB.at[2256,'updated_title'] = "Freaky Friday" # 2141	Freaky Friday (1976)	tt0076054
FL_Disney_IMDB.at[2256,'tconst'] = "tt0076054" # 2141	Freaky Friday (1976)	tt0076054

In [28]:
FL_Disney_IMDB.at[2424,'updated_title'] = "Hannah Montana and Miley Cyrus: Best of Both Worlds Concert" # 2304	Hannah Montana & Miley Cyrus: Best Of Both Wor...	tt1127884
FL_Disney_IMDB.at[2424,'tconst'] = "tt1127884" # 2304	Hannah Montana & Miley Cyrus: Best Of Both Wor... tt1127884

In [29]:
FL_Disney_IMDB.at[1902,'updated_title'] = "Horton Hears a Who!" # Dr. Seuss' Horton Hears A Who tt0451079
FL_Disney_IMDB.at[1902,'tconst'] = "tt0451079" # Dr. Seuss' Horton Hears A Who tt0451079

In [30]:
FL_Disney_IMDB.at[2551,'updated_title'] = "Disney High School Musical: China" # 2429	High School Musical: China	tt1556143
FL_Disney_IMDB.at[2551,'tconst'] = "tt1556143" # 2429	High School Musical: China  tt1556143

In [31]:
FL_Disney_IMDB.at[3065,'updated_title'] = "Jonas Brothers: The 3D Concert Experience" # 2937	Jonas Brothers: The Concert Experience	tt1229827
FL_Disney_IMDB.at[3065,'tconst'] = "tt1229827" # 2937	Jonas Brothers: The Concert Experience  tt1229827

FL_Disney_IMDB.at[3448,'updated_title'] = "Doctor Strange" # 3283	Marvel Studios' Doctor Strange		tt1211837
FL_Disney_IMDB.at[3448,'tconst'] = "tt1211837" # 3283	Marvel Studios' Doctor Strange	  tt1211837

In [32]:
FL_Disney_IMDB.at[3450,'updated_title'] = "Avengers: Age of Ultron" # 3285	Marvel'S Avengers: Age Of Ultron		tt2395427
FL_Disney_IMDB.at[3450,'tconst'] = "tt2395427" # 3285	Marvel'S Avengers: Age Of Ultron	  tt2395427

FL_Disney_IMDB.at[3451,'updated_title'] = "Captain America: The Winter Soldier" # 33286	Marvel'S Captain America: Winter Soldier		tt1843866
FL_Disney_IMDB.at[3451,'tconst'] = "tt1843866" # 3286	Marvel'S Captain America: Winter Soldier	  tt1843866

FL_Disney_IMDB.at[3455,'updated_title'] = "Iron Man 3" # 3290	Marvel'S Iron Man 3 	tt1300854
FL_Disney_IMDB.at[3455,'tconst'] = "tt1300854" # 3290	Marvel'S Iron Man 3	  tt1300854

FL_Disney_IMDB.at[4240,'updated_title'] = "Ralph Breaks the Internet" # 4045	Ralph Breaks The Internet: Wreck-It Ralph 2 	tt5848272
FL_Disney_IMDB.at[4240,'tconst'] = "tt5848272" # 4045	Ralph Breaks The Internet: Wreck-It Ralph 2	  tt5848272

In [33]:
FL_Disney_IMDB.at[4286,'updated_title'] = "Peter Pan 2: Return to Never Land" # 4091	Return To Never Land	tt0280030
FL_Disney_IMDB.at[4286,'tconst'] = "tt0280030" # 4091	Return To Never Land		  tt0280030

FL_Disney_IMDB.at[4664,'updated_title'] = "Sleeping Beauty" # 4465	Sleeping Beauty (1959)	tt0053285tt0053285
FL_Disney_IMDB.at[4664,'tconst'] = "tt0053285" # 4465	Sleeping Beauty (1959)	tt0053285

In [34]:
FL_Disney_IMDB.at[4995,'updated_title'] = "Tall Tale" # 4761	Tall Tale: The Unbelievable Adventure	tt0111359
FL_Disney_IMDB.at[4995,'tconst'] = "tt0111359" # 4761	Tall Tale: The Unbelievable Adventure	tt0111359

FL_Disney_IMDB.at[5262,'updated_title'] = "The Boys" # 5008	The Boys: The Sherman Brothers' Story tt1015971
FL_Disney_IMDB.at[5262,'tconst'] = "tt1015971" # 5008	The Boys: The Sherman Brothers' Story tt1015971

In [35]:

# FL_Disney_IMDB.at[1587,'updated_title'] = "Charlie, the Lonesome Cougar"  #   Charlie The Lonesome Cougar tt0062793
# FL_Disney_IMDB.at[1587,'tconst'] = "tt0062793"  #   Charlie The Lonesome Cougar tt0062793

# FL_Disney_IMDB.at[2363,'updated_title'] = "Herbie Fully Loaded" # 2363	Herbie: Fully Loaded	tt0400497
# FL_Disney_IMDB.at[2363,'tconst'] = "tt0400497" # 2363	Herbie: Fully Loaded  tt0400497

FL_Disney_IMDB.at[6165,'updated_title'] = "Toby Tyler or Ten Weeks with a Circus" # 5891	Toby Tyler	   tt0054390
FL_Disney_IMDB.at[6165,'tconst'] = "tt0054390" # 5891	Toby Tyler	   tt0054390

FL_Disney_IMDB.at[6543,'updated_title'] = "Disneynature: Wings of Life" # 6255	Wings Of Life	  tt1222816
FL_Disney_IMDB.at[6543,'tconst'] = "tt1222816" # 6255	Wings Of Life	tt1222816

FL_Disney_IMDB.at[6544,'updated_title'] = "Winnie the Pooh" # 6256	Winnie The Pooh (2011)	  tt1449283
FL_Disney_IMDB.at[6544,'tconst'] = "tt1449283" # 6256	Winnie The Pooh (2011)	 tt1449283
   

We can see the 22 titles that we've updated

In [36]:
FL_Disney_IMDB.isnull().sum()

title               0
lower_title         0
tconst             60
titleType          82
startYear          82
_merge              0
updated_title    6562
dtype: int64

### 02.06.03 Marking for removal films not found in IMDB that we won't be using

Filling in the tconst with 'remove' so we know to pull these out of the final list later.

In [37]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Disney Animation Collection Volume")), 'tconst'] = 'remove'

In [38]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Disney Sing")), 'tconst'] = 'remove'

In [39]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Disney Princess Sing")), 'tconst'] = 'remove'

In [40]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Mickey Mouse Clubhouse")), 'tconst'] = 'remove'

In [41]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Pixar Short Films Collection")), 'tconst'] = 'remove'

In [42]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Disney Learning Adventures")), 'tconst'] = 'remove'

In [43]:
FL_Disney_IMDB[FL_Disney_IMDB['title'].str.contains("Sing Along Songs")]

title  \
1883  Disney Princess Sing Along Songs Volume One: O...   
1884  Disney Princess Sing Along Songs Volume Three:...   
1885  Disney Princess Sing Along Songs Volume Two: E...   
1889            Disney Sing Along Songs: Happy Haunting   
1890  Disney Sing Along Songs: Home on the Range - L...   
4616  Sing Along Songs: 101 Dalmatians -- Pongo & Pe...   
4617         Sing Along Songs: Brother Bear - On My Way   
4618     Sing Along Songs: Campout At Walt Disney World   
4619  Sing Along Songs: Mary Poppins -- Supercalifra...   
4620        Sing Along Songs: Peter Pan -- You Can Fly!   
4621  Sing Along Songs: The Jungle Book -- The Bare ...   
4622  Sing Along Songs: The Lion King -- Circle Of Life   
6501              Very Merry Christmas Sing Along Songs   

                                            lower_title     tconst titleType  \
1883  disney princess sing along songs volume one on...     remove       NaN   
1884  disney princess sing along songs volume three ...     remove       NaN   
1885  disney princess sing along songs volume two en...     remove       NaN   
1889             disney sing along songs happy haunting     remove       NaN   
1890  disney sing along songs home on the range  lit...     remove     video   
4616    sing along songs 101 dalmatians  pongo  perdita        NaN       NaN   
4617           sing along songs brother bear  on my way  tt1664858     video   
4618      sing along songs campout at walt disney world        NaN       NaN   
4619  sing along songs mary poppins  supercalifragil...        NaN       NaN   
4620            sing along songs peter pan  you can fly        NaN       NaN   
4621  sing along songs the jungle book  the bare nec...        NaN       NaN   
4622     sing along songs the lion king  circle of life        NaN       NaN   
6501              very merry christmas sing along songs        NaN       NaN   

     startYear     _merge updated_title  
1883       NaN  left_only           NaN  
1884       NaN  left_only           NaN  
1885       NaN  left_only           NaN  
1889       NaN  left_only           NaN  
1890      2004       both           NaN  
4616       NaN  left_only           NaN  
4617      2003       both           NaN  
4618       NaN  left_only           NaN  
4619       NaN  left_only           NaN  
4620       NaN  left_only           NaN  
4621       NaN  left_only           NaN  
4622       NaN  left_only           NaN  
6501       NaN  left_only           NaN

In [44]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['title'].str.contains("Sing Along Songs")), 'tconst'] = 'remove'

In [45]:
FL_Disney_IMDB.isnull().sum()

title               0
lower_title         0
tconst             31
titleType          82
startYear          82
_merge              0
updated_title    6562
dtype: int64

In [46]:
FL_Disney_IMDB[FL_Disney_IMDB['tconst'].isnull()].head(35)

title  \
144                   A Very Playhouse Disney Holiday   
318                      Aladdin: The Return of Jafar   
642                                 Atta Girl, Kelly!   
1877                            Disney Junior Holiday   
1881                Disney Princess Party: Volume One   
1898                          Doc McStuffins: Pet Vet   
1986  Elfego Baca And The Swamp Fox, Legendary Heroes   
2326                      Frozen (Sing-Along Edition)   
2425                 Hannah Montana: Pop Star Profile   
2434               Hans Brinker, Or The Silver Skates   
2436           Have Yourself A Goofy Little Christmas   
2552                 High School Musical: The Concert   
2553                   Hilary Duff: The Girl Can Rock   
3449                         Marvel Studios' Eternals   
3512                  Mickey Mouse In Black And White   
3513       Mickey Mouse In Black And White Volume Two   
3560                      Minnie's The Wizard Of Dizz   
3737                                My Dog, The Thief   
4510               Rodgers & Hammerstein's Cinderella   
4527             Rolie Polie Olie: The Baby Bot Chase   
4568       Sheriff Callie's Wild West: Howdy Partner!   
4716             Sofia the First: The Floating Palace   
4964               Stub, The Best Cow Dog in the West   
5199        The Adventures Of Oswald The Lucky Rabbit   
5337   The Emperor's New Groove 2: Kronk's New Groove   
5406                         The Horse Without A Head   
6143   The Wizards Return: Alex vs. Alex (TV Special)   
6169        Tomorrow Land: Disney in Space and Beyond   
6502                                   Vintage Mickey   
6518                      Walt Disney's Funny Factory   
6519                     Walt Disney's Timeless Tales   

                                         lower_title tconst titleType  \
144                  a very playhouse disney holiday    NaN       NaN   
318                      aladdin the return of jafar    NaN       NaN   
642                                  atta girl kelly    NaN       NaN   
1877                           disney junior holiday    NaN       NaN   
1881                disney princess party volume one    NaN       NaN   
1898                          doc mcstuffins pet vet    NaN       NaN   
1986  elfego baca and the swamp fox legendary heroes    NaN       NaN   
2326                        frozen singalong edition    NaN       NaN   
2425                 hannah montana pop star profile    NaN       NaN   
2434               hans brinker or the silver skates    NaN       NaN   
2436          have yourself a goofy little christmas    NaN       NaN   
2552                 high school musical the concert    NaN       NaN   
2553                   hilary duff the girl can rock    NaN       NaN   
3449                         marvel studios eternals    NaN       NaN   
3512                 mickey mouse in black and white    NaN       NaN   
3513      mickey mouse in black and white volume two    NaN       NaN   
3560                      minnies the wizard of dizz    NaN       NaN   
3737                                my dog the thief    NaN       NaN   
4510                rodgers  hammersteins cinderella    NaN       NaN   
4527             rolie polie olie the baby bot chase    NaN       NaN   
4568         sheriff callies wild west howdy partner    NaN       NaN   
4716             sofia the first the floating palace    NaN       NaN   
4964               stub the best cow dog in the west    NaN       NaN   
5199       the adventures of oswald the lucky rabbit    NaN       NaN   
5337     the emperors new groove 2 kronks new groove    NaN       NaN   
5406                        the horse without a head    NaN       NaN   
6143      the wizards return alex vs alex tv special    NaN       NaN   
6169        tomorrow land disney in space and beyond    NaN       NaN   
6502                                  vintage mickey    NaN       NaN   
6518                      walt di

The rest of these are either direct to video or TV movies.  We label them all for removal.

In [47]:
FL_Disney_IMDB.loc[(FL_Disney_IMDB['tconst'].isnull()), 'tconst'] = 'remove'

In [48]:
FL_Disney_IMDB.isnull().sum()

title               0
lower_title         0
tconst              0
titleType          82
startYear          82
_merge              0
updated_title    6562
dtype: int64

In [49]:
FL_Disney_IMDB.drop(FL_Disney_IMDB[FL_Disney_IMDB['tconst'] == "remove"].index, inplace = True)

In [50]:
FL_Disney_IMDB.isnull().sum()

title               0
lower_title         0
tconst              0
titleType          22
startYear          22
_merge              0
updated_title    6497
dtype: int64

We have now removed any films that were not Disney feature releases but there are a lot of duplicates. Let's get the title and tconst columns, then pull in the other IDDB data and explore.

## 02.07 Duplicate Films

### 02.07.01 Identify and Remove Duplicate Films

In [51]:
FL_Disney_IMDB = FL_Disney_IMDB[['title','tconst']]

In [52]:
FL_Disney_IMDB['dup'] = FL_Disney_IMDB['tconst'].duplicated()

There are likely some duplicate tconst values. So we'll remove those.

In [53]:
FL_Disney_IMDB.drop(FL_Disney_IMDB[FL_Disney_IMDB['dup']==True].index, inplace = True)

In [54]:
FL_Disney_IMDB = FL_Disney_IMDB[['title','tconst']]

The index is the original index from our Disney List, 'tconst' is the unique identifier from IMDB.  Now we need to go back to the IMDB data, pull in the meta data, and deal with the duplicates.  

## 02.08 Additional IMDB features

### 02.08.01 Merging to pull in more IMDB features

In [55]:
FL_Disney_IMDB_combined = pd.merge(FL_Disney_IMDB ,                 # left df
                          df_basics,                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

### 02.08.02 Exploring more IMDB features

In [56]:
FL_Disney_IMDB_combined.titleType.value_counts()

tvEpisode       3738
movie            947
short            707
tvMovie          235
video            209
tvSeries         156
videoGame         78
tvMiniSeries      56
tvShort            6
tvSpecial          4
Name: titleType, dtype: int64

In [57]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Deposit")]

title     tconst  titleType           primaryTitle  \
3405  No Deposit No Return  tt0069011      movie  No Deposit, No Return   
3406  No Deposit No Return  tt0074968      movie  No Deposit, No Return   
3407  No Deposit No Return  tt0267784      movie  No Deposit, No Return   
3408  No Deposit No Return  tt0399414      short  No Deposit, No Return   
3409  No Deposit No Return  tt0566167  tvEpisode  No Deposit, No Return   
3410  No Deposit No Return  tt1037519  tvEpisode  No Deposit, No Return   
3411  No Deposit No Return  tt1281796  tvEpisode   No Deposit No Return   
3412  No Deposit No Return  tt1324782  tvEpisode  No Deposit, No Return   
3413  No Deposit No Return  tt2145698      short   No Deposit No Return   
3414  No Deposit No Return  tt8728662      movie   No Deposit No Return   

              originalTitle isAdult startYear endYear runtimeMinutes  \
3405  No Deposit, No Return       0      1972      \N             80   
3406   No Deposit No Return       0      1976      \N            112   
3407  No Deposit, No Return       0      2000      \N             90   
3408  No Deposit, No Return       0      2004      \N             12   
3409  No Deposit, No Return       0      1992      \N             \N   
3410  No Deposit, No Return       0      1965      \N             \N   
3411   No Deposit No Return       0      1990      \N             \N   
3412  No Deposit, No Return       0      1985      \N             \N   
3413   No Deposit No Return       0      1997      \N             22   
3414   No Deposit No Return       1      1975      \N             54   

             genres           lower_title _merge  
3405             \N  no deposit no return   both  
3406  Comedy,Family  no deposit no return   both  
3407         Action  no deposit no return   both  
3408  Romance,Short  no deposit no return   both  
3409         Comedy  no deposit no return   both  
3410         Comedy  no deposit no return   both  
3411         Comedy  no deposit no return   both  
3412         Comedy  no deposit no return   both  
3413    Drama,Short  no deposit no return   both  
3414          Adult  no deposit no return   both

For this project, we want to focus on movies.  Anything tv, video, or short can come out. Keeping only 'movie' types

In [58]:
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined[FL_Disney_IMDB_combined.titleType == 'movie']

In [59]:
FL_Disney_IMDB_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 947 entries, 0 to 6124
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   title           947 non-null    object  
 1   tconst          947 non-null    object  
 2   titleType       947 non-null    object  
 3   primaryTitle    947 non-null    object  
 4   originalTitle   947 non-null    object  
 5   isAdult         947 non-null    object  
 6   startYear       947 non-null    object  
 7   endYear         947 non-null    object  
 8   runtimeMinutes  947 non-null    object  
 9   genres          947 non-null    object  
 10  lower_title     947 non-null    object  
 11  _merge          947 non-null    category
dtypes: category(1), object(11)
memory usage: 89.8+ KB


In [60]:
FL_Disney_IMDB_combined._merge.value_counts()

both          947
left_only       0
right_only      0
Name: _merge, dtype: int64

In [61]:
FL_Disney_IMDB_combined.head(20)

title      tconst titleType  \
0                 101 Dalmatians   tt0115433     movie   
5                 102 Dalmatians   tt0211181     movie   
9   20,000 Leagues Under the Sea   tt0006333     movie   
10  20,000 Leagues Under the Sea   tt0046672     movie   
13  20,000 Leagues Under the Sea   tt0397230     movie   
14  20,000 Leagues Under the Sea   tt0498755     movie   
17  20,000 Leagues Under the Sea  tt10915850     movie   
23  20,000 Leagues Under the Sea   tt9328210     movie   
24                  A Bug's Life   tt0120623     movie   
37             A Christmas Carol   tt0029992     movie   
38             A Christmas Carol   tt0039562     movie   
39             A Christmas Carol   tt0044008     movie   
59             A Christmas Carol   tt0443734     movie   
71             A Christmas Carol  tt10415514     movie   
72             A Christmas Carol  tt10476682     movie   
74             A Christmas Carol   tt1067106     movie   
75             A Christmas Carol  tt10778144     movie   
86             A Christmas Carol  tt13569424     movie   
87             A Christmas Carol  tt13571948     movie   
94             A Christmas Carol  tt16048864     movie   

                    primaryTitle                             originalTitle  \
0                 101 Dalmatians                            101 Dalmatians   
5                 102 Dalmatians                            102 Dalmatians   
9   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
10  20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
13  20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
14  20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
17  20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
23  20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
24                  A Bug's Life                              A Bug's Life   
37             A Christmas Carol                         A Christmas Carol   
38             A Christmas Carol                        Leyenda de Navidad   
39             A Christmas Carol                                   Scrooge   
59             A Christmas Carol  Natale a casa Deejay - A Christmas Carol   
71             A Christmas Carol                         A Christmas Carol   
72             A Christmas Carol                         A Christmas Carol   
74             A Christmas Carol                         A Christmas Carol   
75             A Christmas Carol                         A Christmas Carol   
86             A Christmas Carol                         A Christmas Carol   
87             A Christmas Carol                         A Christmas Carol   
94             A Christmas Carol                         A Christmas Carol   

   isAdult startYear endYear runtimeMinutes                      genres  \
0        0      1996      \N            103      Adventure,Comedy,Crime   
5        0      2000      \N            100     Adventure,Comedy,Family   
9        0      1916      \N            105     Action,Adventure,Sci-Fi   
10       0      1954      \N            127      Adventure,Drama,Family   
13       0      1973      \N             60            Animation,Family   
14       0      1985      \N             50  Action,Adventure,Animation   
17       0        \N      \N             \N                   Adventure   
23       0        \N      \N             \N                   Animation   
24       0      1998      \N             95  Adventure,Animation,Comedy   
37       0      1938      \N             69        Drama,Family,Fantasy   
38       0      1947      \N             80               Drama,Fantasy   
39       0      1951      \N             86        Drama,Family,Fantasy   
59       0      2004      \N             75              Comedy,Fantasy   
71       0        \N      \N             \N                     Fantasy   
72       0      2019      \N             75 

### 02.08.03 Exploring more duplicates

Creating a list of the dupcate value.  Will have to seach IMDB to make sure we have the correct tconst going forward.  We could bump this up against one of the other source lists that include the year of release, but there's no guarantee that everything on this list will be on those, as we already know the counts are different, and there are some discrepancies in the month/year of release. Ultimately, this may prove faster than creating code and trying to account each variant.

In [62]:
dups_df = FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].isin(FL_Disney_IMDB_combined['title'].value_counts()[FL_Disney_IMDB_combined['title'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(541, 12)


title      tconst titleType  \
9    20,000 Leagues Under the Sea   tt0006333     movie   
10   20,000 Leagues Under the Sea   tt0046672     movie   
13   20,000 Leagues Under the Sea   tt0397230     movie   
14   20,000 Leagues Under the Sea   tt0498755     movie   
17   20,000 Leagues Under the Sea  tt10915850     movie   
23   20,000 Leagues Under the Sea   tt9328210     movie   
37              A Christmas Carol   tt0029992     movie   
38              A Christmas Carol   tt0039562     movie   
39              A Christmas Carol   tt0044008     movie   
59              A Christmas Carol   tt0443734     movie   
71              A Christmas Carol  tt10415514     movie   
72              A Christmas Carol  tt10476682     movie   
74              A Christmas Carol   tt1067106     movie   
75              A Christmas Carol  tt10778144     movie   
86              A Christmas Carol  tt13569424     movie   
87              A Christmas Carol  tt13571948     movie   
94              A Christmas Carol  tt16048864     movie   
100             A Christmas Carol   tt2131447     movie   
102             A Christmas Carol   tt2294459     movie   
106             A Christmas Carol   tt4387314     movie   
108             A Christmas Carol   tt4776578     movie   
113             A Christmas Carol   tt5516774     movie   
120             A Christmas Carol   tt7820614     movie   
122             A Christmas Carol   tt8232648     movie   
123             A Christmas Carol   tt9136058     movie   
124             A Christmas Carol   tt9387380     movie   
125             A Christmas Carol   tt9416854     movie   
189                       Aladdin   tt0090601     movie   
190                       Aladdin   tt0103639     movie   
244                       Aladdin   tt6139732     movie   

                     primaryTitle                             originalTitle  \
9    20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
10   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
13   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
14   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
17   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
23   20,000 Leagues Under the Sea              20,000 Leagues Under the Sea   
37              A Christmas Carol                         A Christmas Carol   
38              A Christmas Carol                        Leyenda de Navidad   
39              A Christmas Carol                                   Scrooge   
59              A Christmas Carol  Natale a casa Deejay - A Christmas Carol   
71              A Christmas Carol                         A Christmas Carol   
72              A Christmas Carol                         A Christmas Carol   
74              A Christmas Carol                         A Christmas Carol   
75              A Christmas Carol                         A Christmas Carol   
86              A Christmas Carol                         A Christmas Carol   
87              A Christmas Carol                         A Christmas Carol   
94              A Christmas Carol                         A Christmas Carol   
100             A Christmas Carol                         A Christmas Carol   
102             A Christmas Carol                         A Christmas Carol   
106             A Christmas Carol                         A Christmas Carol   
108             A Christmas Carol                         A Christmas Carol   
113             A Christmas Carol                         A Christmas Carol   
120             A Christmas Carol                         A Christmas Carol   
122             A Christmas Carol                         A Christmas Carol   
123             A Christmas Carol                         A Christmas Carol   
124             A Christmas Carol                         A Christmas Carol   
125             A Christmas Carol                         A Chris

### 02.08.03 Removing the incorrect duplicates

In [63]:
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "20,000 Leagues Under the Sea") & (FL_Disney_IMDB_combined['tconst'] != "tt0046672")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "A Christmas Carol") & (FL_Disney_IMDB_combined['tconst'] != "tt1067106")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Aladdin") & ( (FL_Disney_IMDB_combined['tconst'] != "tt6139732") & (FL_Disney_IMDB_combined['tconst'] != "tt0103639"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Alice in Wonderland") & ( (FL_Disney_IMDB_combined['tconst'] != "tt0043274") & (FL_Disney_IMDB_combined['tconst'] != "tt1014759"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Amy") & (FL_Disney_IMDB_combined['tconst'] != "tt0082017")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Around the World in 80 Days") & (FL_Disney_IMDB_combined['tconst'] != "tt0327437")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Bambi") & (FL_Disney_IMDB_combined['tconst'] != "tt0034492")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Beauty and the Beast") & ( (FL_Disney_IMDB_combined['tconst'] != "tt2771200") & (FL_Disney_IMDB_combined['tconst'] != "tt0101414"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Big Red") & (FL_Disney_IMDB_combined['tconst'] != "tt0055793")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Black Widow") & (FL_Disney_IMDB_combined['tconst'] != "tt3480822")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Bon Voyage!") & (FL_Disney_IMDB_combined['tconst'] != "tt0055807")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Brave") & (FL_Disney_IMDB_combined['tconst'] != "tt1217209")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Cheetah") & (FL_Disney_IMDB_combined['tconst'] != "tt0097053")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Cinderella") & ( (FL_Disney_IMDB_combined['tconst'] != "tt0042332") & (FL_Disney_IMDB_combined['tconst'] != "tt1661199"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Coco") & (FL_Disney_IMDB_combined['tconst'] != "tt2380307")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Dinosaur") & (FL_Disney_IMDB_combined['tconst'] != "tt0130623")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Dumbo") & ( (FL_Disney_IMDB_combined['tconst'] != "tt0033563") & (FL_Disney_IMDB_combined['tconst'] != "tt3861390"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Earth") & (FL_Disney_IMDB_combined['tconst'] != "tt0393597")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Epic") & (FL_Disney_IMDB_combined['tconst'] != "tt0848537")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Emil and the Detectives") & (FL_Disney_IMDB_combined['tconst'] != "tt0058056")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Enchanted") & (FL_Disney_IMDB_combined['tconst'] != "tt0461770")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Endurance") & (FL_Disney_IMDB_combined['tconst'] != "tt0120659")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Fantasia") & (FL_Disney_IMDB_combined['tconst'] != "tt0032455")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Frozen") & (FL_Disney_IMDB_combined['tconst'] != "Frozen")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Gus") & (FL_Disney_IMDB_combined['tconst'] != "tt0074599")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Heidi") & (FL_Disney_IMDB_combined['tconst'] != "tt0107099")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Hercules") & (FL_Disney_IMDB_combined['tconst'] != "tt0119282")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Home on the Range") & (FL_Disney_IMDB_combined['tconst'] != "tt0299172")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Ice Age") & (FL_Disney_IMDB_combined['tconst'] != "tt0268380")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "I'll Be Home for Christmas") & (FL_Disney_IMDB_combined['tconst'] != "tt0155753")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "In Search Of The Castaways") & (FL_Disney_IMDB_combined['tconst'] != "tt0056095")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Inside Out") & (FL_Disney_IMDB_combined['tconst'] != "tt2096673")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Into the Woods") & (FL_Disney_IMDB_combined['tconst'] != "tt2180411")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Invincible") & (FL_Disney_IMDB_combined['tconst'] != "tt0445990")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Iron Will") & (FL_Disney_IMDB_combined['tconst'] != "tt0110157")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Kidnapped") & (FL_Disney_IMDB_combined['tconst'] != "tt0053994")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Luca") & (FL_Disney_IMDB_combined['tconst'] != "tt12801262")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Man of the House") & (FL_Disney_IMDB_combined['tconst'] != "tt0113755")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Midnight Madness") & (FL_Disney_IMDB_combined['tconst'] != "tt0081159")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Miracle") & (FL_Disney_IMDB_combined['tconst'] != "tt0349825	")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Moana") & (FL_Disney_IMDB_combined['tconst'] != "tt3521164")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Mulan") & ( (FL_Disney_IMDB_combined['tconst'] != "tt0120762") & (FL_Disney_IMDB_combined['tconst'] != "tt4566758"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Never a Dull Moment") & (FL_Disney_IMDB_combined['tconst'] != "tt0063341")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "No Deposit No Return") & (FL_Disney_IMDB_combined['tconst'] != "tt0074968")].index)
# *************************************************************************  This is a TV movie, and would have been removed at this point
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Oliver Twist") & (FL_Disney_IMDB_combined['tconst'] != "tt0119825")].index)
# *************************************************************************
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Peter Pan") & (FL_Disney_IMDB_combined['tconst'] != "tt0046183")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Pinocchio") & (FL_Disney_IMDB_combined['tconst'] != "tt0032910")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Rascal") & (FL_Disney_IMDB_combined['tconst'] != "tt0064875")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Rio") & (FL_Disney_IMDB_combined['tconst'] != "tt1436562")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Robin Hood") & (FL_Disney_IMDB_combined['tconst'] != "tt0070608")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "RocketMan") & (FL_Disney_IMDB_combined['tconst'] != "tt0120029")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Sky High") & (FL_Disney_IMDB_combined['tconst'] != "tt0405325")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Smith!") & (FL_Disney_IMDB_combined['tconst'] != "tt0065003")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Snow White and the Seven Dwarfs") & (FL_Disney_IMDB_combined['tconst'] != "tt0029583")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Something Wicked This Way Comes") & (FL_Disney_IMDB_combined['tconst'] != "tt0086336")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Soul") & (FL_Disney_IMDB_combined['tconst'] != "tt2948372")].index)
# ************************************************************************* This is a TV movie, and would have been removed at this point
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Starstruck") & (FL_Disney_IMDB_combined['tconst'] != "tt1579247")].index)
# *************************************************************************
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Swiss Family Robinson") & (FL_Disney_IMDB_combined['tconst'] != "tt0054357")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Tangled") & (FL_Disney_IMDB_combined['tconst'] != "tt0398286")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Teacher's Pet") & (FL_Disney_IMDB_combined['tconst'] != "tt0350194")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Tex") & (FL_Disney_IMDB_combined['tconst'] != "tt0084783")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Black Hole") & (FL_Disney_IMDB_combined['tconst'] != "tt0078869")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Duke") & (FL_Disney_IMDB_combined['tconst'] != "tt0196516")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Hunchback of Notre Dame") & (FL_Disney_IMDB_combined['tconst'] != "tt0116583")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Kid") & (FL_Disney_IMDB_combined['tconst'] != "tt0219854")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Lion King") & ( (FL_Disney_IMDB_combined['tconst'] != "tt0110357") & (FL_Disney_IMDB_combined['tconst'] != "tt6105098"))].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Little Mermaid") & (FL_Disney_IMDB_combined['tconst'] != "tt0097757")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Lone Ranger") & (FL_Disney_IMDB_combined['tconst'] != "tt1210819")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Rookie") & (FL_Disney_IMDB_combined['tconst'] != "tt0265662")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Sorcerer's Apprentice") & (FL_Disney_IMDB_combined['tconst'] != "tt0963966")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Three Musketeers") & (FL_Disney_IMDB_combined['tconst'] != "tt0108333")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Wild") & (FL_Disney_IMDB_combined['tconst'] != "tt0405469")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Tonka") & (FL_Disney_IMDB_combined['tconst'] != "tt0052300")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Treasure Island") & (FL_Disney_IMDB_combined['tconst'] != "tt0043067")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Underdog") & (FL_Disney_IMDB_combined['tconst'] != "tt0467110")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Valiant") & (FL_Disney_IMDB_combined['tconst'] != "tt0361089")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "White Fang") & (FL_Disney_IMDB_combined['tconst'] != "tt0103247")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Heavyweights") & (FL_Disney_IMDB_combined['tconst'] != "tt0110006")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Old Dogs") & (FL_Disney_IMDB_combined['tconst'] != "tt0976238")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Pocahontas") & (FL_Disney_IMDB_combined['tconst'] != "tt0114148")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Hocus Pocus") & (FL_Disney_IMDB_combined['tconst'] != "tt0107120")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Prom") & (FL_Disney_IMDB_combined['tconst'] != "tt1604171")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The BFG") & (FL_Disney_IMDB_combined['tconst'] != "tt3691740")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Tomorrowland") & (FL_Disney_IMDB_combined['tconst'] != "tt1964418")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Black Panther") & (FL_Disney_IMDB_combined['tconst'] != "tt1825683")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Bolt") & (FL_Disney_IMDB_combined['tconst'] != "tt0397892")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Music Man")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Miracle Worker")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "The Perfect Game")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Tower Of Terror")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['title'] == "Bedtime Stories")].index)

### 02.08.05 Exploring films prior to 1937

There shouldn't be anything in here prios to 1937 (when Snow White and the Seven Dwarfs was released) and I wouldn't expect to see any startYear values missing unless it's a future release.  So lets explore all of those.

In [64]:
FL_Disney_IMDB_combined.sort_values(by='startYear')

title      tconst titleType  \
3787                        Pollyanna   tt0011588     movie   
4183                      Shipwrecked   tt0017379     movie   
4301  Snow White and the Seven Dwarfs   tt0029583     movie   
4822                The Biscuit Eater   tt0032254     movie   
1785                         Fantasia   tt0032455     movie   
...                               ...         ...       ...   
4056                           Robots  tt12579470     movie   
4275                      Smart House   tt6824720     movie   
4802                   The Aristocats  tt17220728     movie   
4883               The Christmas Star   tt3511874     movie   
4989    The Greatest Game Ever Played  tt16358988     movie   

                         primaryTitle                    originalTitle  \
3787                        Pollyanna                        Pollyanna   
4183                      Shipwrecked                      Shipwrecked   
4301  Snow White and the Seven Dwarfs  Snow White and the Seven Dwarfs   
4822                The Biscuit Eater                The Biscuit Eater   
1785                         Fantasia                         Fantasia   
...                               ...                              ...   
4056                           Robots                           Robots   
4275                      Smart House                      Smart House   
4802                   The Aristocats                   The Aristocats   
4883               The Christmas Star               The Christmas Star   
4989    The Greatest Game Ever Played    The Greatest Game Ever Played   

     isAdult startYear endYear runtimeMinutes                      genres  \
3787       0      1920      \N             58         Comedy,Drama,Family   
4183       0      1926      \N             74     Adventure,Drama,Romance   
4301       0      1937      \N             83  Adventure,Animation,Family   
4822       0      1940      \N             81                Drama,Family   
1785       0      1940      \N            125    Animation,Family,Fantasy   
...      ...       ...     ...            ...                         ...   
4056       0        \N      \N             \N       Comedy,Romance,Sci-Fi   
4275       0        \N      \N             \N             Horror,Thriller   
4802       0        \N      \N             \N                      Family   
4883       0        \N      \N             \N                Drama,Family   
4989       0        \N      \N             \N                 Documentary   

                          lower_title _merge  
3787                        pollyanna   both  
4183                      shipwrecked   both  
4301  snow white and the seven dwarfs   both  
4822                the biscuit eater   both  
1785                         fantasia   both  
...                               ...    ...  
4056                           robots   both  
4275                      smart house   both  
4802                   the aristocats   both  
4883               the christmas star   both  
4989    the greatest game ever played   both  

[466 rows x 12 columns]

In [66]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title']== 'Pollyanna']

title     tconst titleType primaryTitle originalTitle isAdult  \
3787  Pollyanna  tt0011588     movie    Pollyanna     Pollyanna       0   
3788  Pollyanna  tt0054195     movie    Pollyanna     Pollyanna       0   

     startYear endYear runtimeMinutes               genres lower_title _merge  
3787      1920      \N             58  Comedy,Drama,Family   pollyanna   both  
3788      1960      \N            134  Comedy,Drama,Family   pollyanna   both

In [67]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title']== 'Shipwrecked']

title     tconst titleType primaryTitle     originalTitle isAdult  \
4183  Shipwrecked  tt0017379     movie  Shipwrecked       Shipwrecked       0   
4184  Shipwrecked  tt0099816     movie  Shipwrecked  Haakon Haakonsen       0   

     startYear endYear runtimeMinutes                   genres  lower_title  \
4183      1926      \N             74  Adventure,Drama,Romance  shipwrecked   
4184      1990      \N             92         Adventure,Family  shipwrecked   

     _merge  
4183   both  
4184   both

In [73]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains('Alexander')]

title      tconst titleType  \
256  Alexander and the Terrible, Horrible, No Good,...  tt13782222     movie   
258  Alexander and the Terrible, Horrible, No Good,...   tt1698641     movie   

                                          primaryTitle  \
256  Alexander and the Terrible, Horrible, No Good,...   
258  Alexander and the Terrible, Horrible, No Good,...   

                                         originalTitle isAdult startYear  \
256  Alexander and the Terrible, Horrible, No Good,...       0        \N   
258  Alexander and the Terrible, Horrible, No Good,...       0      2014   

    endYear runtimeMinutes               genres  \
256      \N             \N        Comedy,Family   
258      \N             81  Comedy,Drama,Family   

                                           lower_title _merge  
256  alexander and the terrible horrible no good ve...   both  
258  alexander and the terrible horrible no good ve...   both

In [74]:
# Alexander and the Terrible, Horrible, No Good,... is a remake under development
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt13782222"].index, inplace = True)

In [75]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains('Encanto')]

title     tconst titleType primaryTitle originalTitle isAdult  \
1693  Encanto  tt2953050     movie      Encanto       Encanto       0   
1695  Encanto  tt5964462     movie      Encanto       Encanto       0   

     startYear endYear runtimeMinutes                   genres lower_title  \
1693      2021      \N            102  Animation,Comedy,Family     encanto   
1695        \N      \N             \N                    Drama     encanto   

     _merge  
1693   both  
1695   both

In [76]:
#tt5964462 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt5964462"].index, inplace = True)

In [77]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains('Flight of the Navigator')]

title     tconst titleType             primaryTitle  \
1870  Flight of the Navigator  tt0091059     movie  Flight of the Navigator   
1872  Flight of the Navigator  tt1444308     movie  Flight of the Navigator   

                originalTitle isAdult startYear endYear runtimeMinutes  \
1870  Flight of the Navigator       0      1986      \N             90   
1872  Flight of the Navigator       0        \N      \N             \N   

                       genres              lower_title _merge  
1870  Adventure,Comedy,Family  flight of the navigator   both  
1872  Adventure,Comedy,Family  flight of the navigator   both

In [78]:
#tt1444308 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt1444308"].index, inplace = True)

In [79]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains('James and the')]

title     tconst titleType  \
2697  James and the Giant Peach  tt0116683     movie   
2699  James and the Giant Peach  tt6009922     movie   

                   primaryTitle              originalTitle isAdult startYear  \
2697  James and the Giant Peach  James and the Giant Peach       0      1996   
2699  James and the Giant Peach  James and the Giant Peach       0        \N   

     endYear runtimeMinutes                      genres  \
2697      \N             79  Adventure,Animation,Family   
2699      \N             \N    Adventure,Family,Fantasy   

                    lower_title _merge  
2697  james and the giant peach   both  
2699  james and the giant peach   both

In [80]:
#tt6009922 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt6009922"].index, inplace = True)

In [81]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Mr. Toad's Wild Ride")]

title     tconst titleType          primaryTitle  \
3313  Mr. Toad's Wild Ride  tt0118172     movie  Mr. Toad's Wild Ride   
3314  Mr. Toad's Wild Ride  tt2369119     movie  Mr. Toad's Wild Ride   

                originalTitle isAdult startYear endYear runtimeMinutes  \
3313  The Wind in the Willows       0      1996      \N             88   
3314     Mr. Toad's Wild Ride       0        \N      \N             \N   

                       genres         lower_title _merge  
3313  Adventure,Comedy,Family  mr toads wild ride   both  
3314         Animation,Family  mr toads wild ride   both

In [82]:
#tt2369119 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt2369119"].index, inplace = True)

In [83]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Robots")]

title      tconst titleType primaryTitle originalTitle isAdult  \
4042  Robots   tt0358082     movie       Robots        Robots       0   
4056  Robots  tt12579470     movie       Robots        Robots       0   

     startYear endYear runtimeMinutes                      genres lower_title  \
4042      2005      \N             91  Adventure,Animation,Comedy      robots   
4056        \N      \N             \N       Comedy,Romance,Sci-Fi      robots   

     _merge  
4042   both  
4056   both

In [84]:
#tt12579470 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt12579470"].index, inplace = True)

In [85]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Smart House")]

title     tconst titleType primaryTitle originalTitle isAdult  \
4275  Smart House  tt6824720     movie  Smart House   Smart House       0   

     startYear endYear runtimeMinutes           genres  lower_title _merge  
4275        \N      \N             \N  Horror,Thriller  smart house   both

In [86]:
#tt6824720 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt6824720"].index, inplace = True)

In [87]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Aristocats")]

title      tconst titleType    primaryTitle   originalTitle  \
4801  The Aristocats   tt0065421     movie  The Aristocats  The AristoCats   
4802  The Aristocats  tt17220728     movie  The Aristocats  The Aristocats   

     isAdult startYear endYear runtimeMinutes                      genres  \
4801       0      1970      \N             78  Adventure,Animation,Comedy   
4802       0        \N      \N             \N                      Family   

         lower_title _merge  
4801  the aristocats   both  
4802  the aristocats   both

In [88]:
#tt6824720 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt17220728"].index, inplace = True)

In [89]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("The Christmas Star")]

title     tconst titleType        primaryTitle  \
4879  The Christmas Star  tt0090840     movie  The Christmas Star   
4883  The Christmas Star  tt3511874     movie  The Christmas Star   

           originalTitle isAdult startYear endYear runtimeMinutes  \
4879  The Christmas Star       0      1986      \N             94   
4883  The Christmas Star       0        \N      \N             \N   

            genres         lower_title _merge  
4879  Drama,Family  the christmas star   both  
4883  Drama,Family  the christmas star   both

In [90]:
#tt3511874 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt3511874"].index, inplace = True)

In [91]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Greatest Game Ever")]

title      tconst titleType  \
4986  The Greatest Game Ever Played   tt0388980     movie   
4989  The Greatest Game Ever Played  tt16358988     movie   

                       primaryTitle                  originalTitle isAdult  \
4986  The Greatest Game Ever Played  The Greatest Game Ever Played       0   
4989  The Greatest Game Ever Played  The Greatest Game Ever Played       0   

     startYear endYear runtimeMinutes                 genres  \
4986      2005      \N            120  Biography,Drama,Sport   
4989        \N      \N             \N            Documentary   

                        lower_title _merge  
4986  the greatest game ever played   both  
4989  the greatest game ever played   both

In [92]:
#tt3511874 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt16358988"].index, inplace = True)

In [93]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Tower Of Terror")]

Empty DataFrame
Columns: [title, tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres, lower_title, _merge]
Index: []

In [94]:
#tt4922444 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt4922444"].index, inplace = True)

In [95]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].str.contains("Tron")]

title      tconst titleType primaryTitle originalTitle isAdult startYear  \
5939  Tron   tt0084827     movie         TRON          TRON       0      1982   
5942  Tron  tt12916062     movie         Tron          Tron       0        \N   

     endYear runtimeMinutes                   genres lower_title _merge  
5939      \N             96  Action,Adventure,Sci-Fi        tron   both  
5942      \N             \N                       \N        tron   both

In [96]:
#tt12916062 is an error
FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['tconst'] == "tt12916062"].index, inplace = True)

In [97]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['startYear'] == "\\N"]

Empty DataFrame
Columns: [title, tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres, lower_title, _merge]
Index: []

In [72]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['title'].isnull()]

Empty DataFrame
Columns: [title, tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres, lower_title, _merge]
Index: []

In [99]:
FL_Disney_IMDB_combined.sort_values(by='titleType')
FL_Disney_IMDB_combined['titleType'].value_counts()

movie    453
Name: titleType, dtype: int64

### 02.08.06 Removing films prior to 1937

In [100]:
FL_Disney_IMDB_combined.sort_values(by='startYear')
FL_Disney_IMDB_combined['startYear'].value_counts()

2009    13
2016    12
2007    12
2010    12
2011    11
        ..
1937     1
1942     1
1946     1
1947     1
1944     1
Name: startYear, Length: 83, dtype: int64

In [69]:
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['tconst'] == "tt0011588")].index)
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['tconst'] == "tt0017379")].index)

In [71]:
FL_Disney_IMDB_combined = FL_Disney_IMDB_combined.drop(FL_Disney_IMDB_combined[(FL_Disney_IMDB_combined['tconst'] == "tt0246786")].index)

In [70]:
FL_Disney_IMDB_combined[FL_Disney_IMDB_combined['startYear'] == "\\N"]

title      tconst titleType  \
256   Alexander and the Terrible, Horrible, No Good,...  tt13782222     movie   
1695                                            Encanto   tt5964462     movie   
1872                            Flight of the Navigator   tt1444308     movie   
2699                          James and the Giant Peach   tt6009922     movie   
3314                               Mr. Toad's Wild Ride   tt2369119     movie   
4056                                             Robots  tt12579470     movie   
4275                                        Smart House   tt6824720     movie   
4802                                     The Aristocats  tt17220728     movie   
4883                                 The Christmas Star   tt3511874     movie   
4989                      The Greatest Game Ever Played  tt16358988     movie   
5942                                               Tron  tt12916062     movie   

                                           primaryTitle  \
256   Alexander and the Terrible, Horrible, No Good,...   
1695                                            Encanto   
1872                            Flight of the Navigator   
2699                          James and the Giant Peach   
3314                               Mr. Toad's Wild Ride   
4056                                             Robots   
4275                                        Smart House   
4802                                     The Aristocats   
4883                                 The Christmas Star   
4989                      The Greatest Game Ever Played   
5942                                               Tron   

                                          originalTitle isAdult startYear  \
256   Alexander and the Terrible, Horrible, No Good,...       0        \N   
1695                                            Encanto       0        \N   
1872                            Flight of the Navigator       0        \N   
2699                          James and the Giant Peach       0        \N   
3314                               Mr. Toad's Wild Ride       0        \N   
4056                                             Robots       0        \N   
4275                                        Smart House       0        \N   
4802                                     The Aristocats       0        \N   
4883                                 The Christmas Star       0        \N   
4989                      The Greatest Game Ever Played       0        \N   
5942                                               Tron       0        \N   

     endYear runtimeMinutes                    genres  \
256       \N             \N             Comedy,Family   
1695      \N             \N                     Drama   
1872      \N             \N   Adventure,Comedy,Family   
2699      \N             \N  Adventure,Family,Fantasy   
3314      \N             \N          Animation,Family   
4056      \N             \N     Comedy,Romance,Sci-Fi   
4275      \N             \N           Horror,Thriller   
4802      \N             \N                    Family   
4883      \N             \N              Drama,Family   
4989      \N             \N               Documentary   
5942      \N             \N                        \N   

                                            lower_title _merge  
256   alexander and the terrible horrible no good ve...   both  
1695                                            encanto   both  
1872                            flight of the navigator   both  
2699                          james and the giant peach   both  
3314                                 mr toads wild ride   both  
4056                                             robots   both  
4275                                        smart house   both  
4802                                     the aristocats   both  
4883                                 the christmas star   both  
4989                      the greatest game ever played   both  
5942                                       

In [101]:
FL_Disney_IMDB_combined.sort_values(by='genres')
FL_Disney_IMDB_combined['genres'].value_counts()

Adventure,Animation,Comedy    75
Adventure,Comedy,Family       21
Adventure,Family,Fantasy      20
Drama,Family                  19
Action,Adventure,Comedy       19
                              ..
Adventure,Family,History       1
Comedy,Western                 1
Musical,Romance                1
Adventure,Documentary          1
Documentary,Family,News        1
Name: genres, Length: 75, dtype: int64

In [98]:
FL_Disney_IMDB_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453 entries, 0 to 6124
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   title           453 non-null    object  
 1   tconst          453 non-null    object  
 2   titleType       453 non-null    object  
 3   primaryTitle    453 non-null    object  
 4   originalTitle   453 non-null    object  
 5   isAdult         453 non-null    object  
 6   startYear       453 non-null    object  
 7   endYear         453 non-null    object  
 8   runtimeMinutes  453 non-null    object  
 9   genres          453 non-null    object  
 10  lower_title     453 non-null    object  
 11  _merge          453 non-null    category
dtypes: category(1), object(11)
memory usage: 43.0+ KB


We've removed errors, corrected erros, removed duplcates, and removed anything that was not a feature release.  That leaves us with 453 Disney Movies. This is the data we will start with to gather Ratings and Votes

## 02.09 Export Features Films

In [102]:
FL_Disney_IMDB_combined.to_csv('../Bens_Data/Combined_IMDB_Disney_453.csv')